In [4]:
import numpy as np
import pandas as pd

In [25]:
#root_path = 'c:/Users/jiaxihe/Desktop/MelbDatathon2017/'
root_path ='../'
chronic_illness_lookup = pd.read_csv(root_path + 'Lookups/ChronicIllness_LookUp.txt', sep='\t')

In [32]:
diabetes_drugs = set(chronic_illness_lookup.MasterProductID[chronic_illness_lookup.ChronicIllness == 'Diabetes'])
print(np.sort(list(diabetes_drugs)))

[  140   141   142   387   388   389   390   579   580   581   850   851
   852   853   859   860   887   888   889   890  1182  1183  2253  2427
  2428  2430  2431  2432  2433  2434  2435  2436  2437  2438  2439  2440
  2443  2444  2445  2446  2447  2448  2449  2596  2597  2598  2599  3183
  3509  3510  3511  3512  3513  3773  3786  3787  3789  3791  3792  3793
  3794  3795  3796  3797  3798  3799  3800  3801  3802  3803  3804  3805
  3817  3827  3828  3833  3834  3835  3836  3837  3838  3854  3855  3856
  3863  3864  4068  4070  4071  4072  4077  4079  4080  4086  4087  4088
  4157  4158  4161  4163  4231  4232  4234  4236  4237  4238  4241  4243
  4244  4248  4280  4299  4476  4477  4478  4479  4480  4481  4721  4722
  4723  4800  4801  5195  5199  5275  5279  5280  5281  5282  5283  5284
  5285  5286  5287  5288  5289  5290  5291  5292  5293  5294  5295  5296
  5297  5298  5299  5300  5301  5302  5303  5359  5471  5472  5927  6043
  6044  6045  6046  6047  6048  6049  6050  6051  6

In [34]:
drug_lookup = pd.read_csv(root_path + 'Lookups/Drug_LookUp.txt', sep='\t')
diabetes_drugs2 = set(drug_lookup.MasterProductID[drug_lookup.ATCLevel2Code == 'A10'])  # 13533 missing
print(np.sort(list(diabetes_drugs2)))

[  140   141   142   387   388   389   390   579   580   581   850   851
   852   853   859   860   887   888   889   890  1182  1183  2253  2427
  2428  2430  2431  2432  2433  2434  2435  2436  2437  2438  2439  2440
  2443  2444  2445  2446  2447  2448  2449  2596  2597  2598  2599  3183
  3509  3510  3511  3512  3513  3773  3786  3787  3789  3791  3792  3793
  3794  3795  3796  3797  3798  3799  3800  3801  3802  3803  3804  3805
  3817  3827  3828  3833  3834  3835  3836  3837  3838  3854  3855  3856
  3863  3864  4068  4070  4071  4072  4077  4079  4080  4086  4087  4088
  4157  4158  4161  4163  4231  4232  4234  4236  4237  4238  4241  4243
  4244  4248  4280  4299  4476  4477  4478  4479  4480  4481  4721  4722
  4723  4800  4801  5195  5199  5275  5279  5280  5281  5282  5283  5284
  5285  5286  5287  5288  5289  5290  5291  5292  5293  5294  5295  5296
  5297  5298  5299  5300  5301  5302  5303  5359  5471  5472  5927  6043
  6044  6045  6046  6047  6048  6049  6050  6051  6

In [44]:
diabetes_drugs-diabetes_drugs2

{7137, 13533}

In [88]:
diabetes_drugs2-diabetes_drugs

{7819}

In [93]:
diabetes_drugs=diabetes_drugs | diabetes_drugs2
print(np.sort(list(diabetes_drugs)))

[  140   141   142   387   388   389   390   579   580   581   850   851
   852   853   859   860   887   888   889   890  1182  1183  2253  2427
  2428  2430  2431  2432  2433  2434  2435  2436  2437  2438  2439  2440
  2443  2444  2445  2446  2447  2448  2449  2596  2597  2598  2599  3183
  3509  3510  3511  3512  3513  3773  3786  3787  3789  3791  3792  3793
  3794  3795  3796  3797  3798  3799  3800  3801  3802  3803  3804  3805
  3817  3827  3828  3833  3834  3835  3836  3837  3838  3854  3855  3856
  3863  3864  4068  4070  4071  4072  4077  4079  4080  4086  4087  4088
  4157  4158  4161  4163  4231  4232  4234  4236  4237  4238  4241  4243
  4244  4248  4280  4299  4476  4477  4478  4479  4480  4481  4721  4722
  4723  4800  4801  5195  5199  5275  5279  5280  5281  5282  5283  5284
  5285  5286  5287  5288  5289  5290  5291  5292  5293  5294  5295  5296
  5297  5298  5299  5300  5301  5302  5303  5359  5471  5472  5927  6043
  6044  6045  6046  6047  6048  6049  6050  6051  6

In [68]:
def read_patients(i):
    return pd.read_csv(root_path + 'Transactions/patients_%d.txt' % i, sep='\t',  
                       parse_dates = ['Prescription_Week', 'Dispense_Week'])

In [ ]:
year2016 = np.datetime64('2016-01-01')

matrix = []
for i in range(1, 51):
    print('process file %d' % i)
    patients_data = read_patients(i)

    for patient_ID, patient in patients_data.groupby('Patient_ID'):
        pre2015 = patient[patient.Dispense_Week < year2016]
        post2015 = patient[patient.Dispense_Week >= year2016]
        
        prediction  = (len(set(pre2015.Drug_ID)&diabetes_drugs) != 0)
        groud_truth = (len(set(post2015.Drug_ID)&diabetes_drugs) != 0)
        
        matrix.append((patient_ID, prediction, groud_truth))

In [77]:
patients_data = read_patients(25)    

In [106]:
patients_data.groupby('Patient_ID')

In [107]:
set(pat270749.Drug_ID)&diabetes_drugs

{140, 2440, 4476, 9885, 9975, 9999, 10450, 11538, 12579}

In [109]:
#Take the patient 270749 as an example to see the frequency diepense of diabetes drug.
pat270749 = patients_data[patients_data.Patient_ID == 270749]
diabete_drug270749 = set(pat270749.Drug_ID)&diabetes_drugs
pat270749[pat270749.Drug_ID.isin(diabete_drug270749)]

,Patient_ID,Store_ID,Prescriber_ID,Drug_ID,SourceSystem_Code,Prescription_Week,Dispense_Week,Drug_Code,NHS_Code,IsDeferredScript,Script_Qty,Dispensed_Qty,MaxDispense_Qty,PatientPrice_Amt,WholeSalePrice_Amt,GovernmentReclaim_Amt,RepeatsTotal_Qty,RepeatsLeft_Qty,StreamlinedApproval_Code
314309,270749,2110,56090,140,F,2010-08-29,2011-01-16,ACTO3,8694N,0,28,3,NaN,5.6,50.09,55.92,6,3,NaN
314310,270749,2110,56090,140,F,2010-08-29,2011-02-13,ACTO3,8694N,0,28,4,NaN,5.6,50.09,55.92,6,2,NaN
314319,270749,2110,56090,140,F,2011-05-01,2011-06-12,ACTO3,8694N,0,28,2,NaN,5.6,42.08,47.40,6,4,NaN
314321,270749,2110,56090,140,F,2011-05-01,2011-07-10,ACTO3,8694N,0,28,3,NaN,5.6,42.08,47.40,6,3,NaN
314322,270749,2110,56090,140,F,2011-05-01,2011-08-07,ACTO3,8694N,0,28,4,NaN,5.6,42.08,47.40,6,2,NaN
314332,270749,1247,46407,9999,F,2011-07-17,2011-11-20,PISZ1,8694N,0,28,2,NaN,0.0,42.08,53.00,6,4,NaN
314333,270749,2110,46407,9975,F,2011-07-17,2011-12-11,VEXA1,8694N,0,28,3,NaN,0.0,42.08,53.00,6,3,NaN
314334,270749,2110,46407,9885,F,2011-07-17,2012-01-22,PIOG5,8694N,0,28,4,NaN,5.8,42.08,47.20,6,2,3540.0
314342,270749,2110,56090,9885,F,2011-12-04,2012-05-27,PIOG5,8694N,0,28,2,NaN,5.8,42.08,47.20,6,4,3541.0
314345,270749,2110,56090,9885,F,2011-12-04,2012-06-17,PIOG5,8694N,0,28,3,NaN,5.8,42.08,47.20,6,3,3541.0


In [122]:
year2015mid = np.datetime64('2015-09-01')
pat270749[pat270749.Drug_ID.isin(diabete_drug270749) & (pat270749.Dispense_Week > year2016) & (pat270749.Dispense_Week > year2015mid)]

,Patient_ID,Store_ID,Prescriber_ID,Drug_ID,SourceSystem_Code,Prescription_Week,Dispense_Week,Drug_Code,NHS_Code,IsDeferredScript,Script_Qty,Dispensed_Qty,MaxDispense_Qty,PatientPrice_Amt,WholeSalePrice_Amt,GovernmentReclaim_Amt,RepeatsTotal_Qty,RepeatsLeft_Qty,StreamlinedApproval_Code
314443,270749,72,10314,2440,F,2015-04-19,2015-09-06,DIAF6,9435N,0,120,4,NaN,0.0,4.81,15.23,6,2,NaN
314458,270749,72,10314,11538,F,2015-12-06,2016-02-28,METF48,9435N,0,120,2,NaN,6.2,4.81,9.03,6,4,NaN
314464,270749,72,10314,11538,F,2015-12-06,2016-05-08,METF48,9435N,0,120,3,NaN,6.2,4.81,9.03,6,3,NaN
314472,270749,72,48517,12579,F,2016-07-03,2016-07-24,ARDI1,9302N,0,60,2,NaN,0.0,7.24,17.80,6,4,NaN
314474,270749,72,48517,12579,F,2016-07-03,2016-08-28,ARDI1,9302N,0,60,3,NaN,0.0,7.24,17.80,6,3,NaN
314478,270749,72,48517,4476,F,2016-07-31,2016-08-28,JANU1,9182G,0,28,2,NaN,0.0,45.29,55.85,6,4,5655.0
314480,270749,72,48517,12579,F,2016-07-03,2016-09-25,ARDI1,9302N,0,60,4,NaN,0.0,7.24,17.80,6,2,NaN
314489,270749,72,48517,12579,F,2016-11-27,2016-12-25,ARDI1,9302N,0,60,2,NaN,0.0,7.24,17.80,6,4,NaN
314492,270749,72,48517,12579,F,2016-11-27,2017-02-19,ARDI1,9302N,0,60,3,NaN,6.3,7.24,11.50,6,3,NaN
314494,270749,72,48517,12579,F,2016-11-27,2017-03-12,ARDI1,9302N,0,60,4,NaN,6.3,7.24,11.50,6,2,NaN


In [ ]:
year2016 = np.datetime64('2016-01-01')
year2015 = np.datetime64('2015-01-01')

# only see the people who take durgs from time period 2015-09-01 to 2016-01-01
matrix = []
for i in range(1, 51):
    print('process file %d' % i)
    patients_data = read_patients(i)

    for patient_ID, patient in patients_data.groupby('Patient_ID'):
        within2015 = patient[(patient.Dispense_Week < year2016) & (patient.Dispense_Week > year2015)]
        post2015 = patient[patient.Dispense_Week >= year2016]
        
        prediction  = (len(set(within2015.Drug_ID)&diabetes_drugs) != 0)
        groud_truth = (len(set(post2015.Drug_ID)&diabetes_drugs) != 0)
        
        matrix.append((patient_ID, prediction, groud_truth))

process file 1


In [125]:
matrix = np.asarray(matrix)

In [127]:
from sklearn.metrics import classification_report, roc_auc_score

train = matrix[:279200]
test  = matrix[279200:]

print(roc_auc_score(train[:, 2], train[:, 1]))
print(classification_report(train[:, 2], train[:, 1]))

0.878837012889
             precision    recall  f1-score   support

          0       0.95      0.99      0.97    225550
          1       0.94      0.77      0.85     53650

avg / total       0.95      0.95      0.94    279200



In [ ]:
with open('baseline.csv', 'w+') as fout:
    fout.write('Patient_ID,Diabetes\n')
    for patient in test:
        fout.write('%d,%d\n' % (patient[0], patient[1]))